In [1]:
import json
import re
import csv
import pandas as pd
import tqdm
import numpy as np

In [ ]:
main_col=['_id', 'title', 'year',  'n_citation', 'lang',
         'page_start', 'page_end', 'volume', 'issue', 'issn',
       'isbn', 'doi', 'pdf']
att_dict=['page_start', 'page_end', 'volume', 'issue', 'issn',
       'isbn', 'doi', 'pdf', 'url']
sub_col=[ 'fos', 'keywords',  'venue', 'references']
abstract_col=['_id', 'abstract']
authors_col=['_id', 'article_id', 'avatar', 'bio', 'email', 'gid', 'homepage', 'name', 'name_zh', 'oid', 'oid_zh'
             , 'orcid', 'org', 'org_zh', 'orgid', 'orgs: st', 'orgs_zh: st', 'position', 'sid']

In [222]:
#данные первого уровня по статьям без abstract

df=[]
cnt=0
pd.DataFrame([], columns=main_col).to_csv('main_table.csv', index=False)
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in f:
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            df.append(json.loads(cur_json[:-2]))
            cur_json=''
            cnt+=1
            if cnt==500000:
                df=pd.DataFrame(df)
                df[main_col].to_csv(r'main_table.csv', mode='a', index=False, header=False)
                df=[]
                cnt=0
        if line == '}\n':
            df.append(json.loads(cur_json))
            df=pd.DataFrame(df)
            df[main_col].to_csv(r'main_table.csv', mode='a', index=False, header=False)

In [249]:
#abstract
df=[]
cnt=0
pd.DataFrame([], columns=add).to_csv('abstract.csv', index=False, quoting=csv.QUOTE_NONE, escapechar='|')
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in f:
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            df.append(json.loads(cur_json[:-2]))
            cur_json=''
            cnt+=1
            if cnt==500000:
                df=pd.DataFrame(df)
                df[add].to_csv(r'abstract.csv', mode='a', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='|')
                df=[]
                cnt=0
        if line == '}\n':
            df.append(json.loads(cur_json))
            df=pd.DataFrame(df)
            df[add].to_csv(r'abstract.csv', mode='a', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='|')


In [60]:
#abstract without none
d=pd.read_csv('abstract.csv', quoting=csv.QUOTE_NONE,escapechar='|')
d=d.dropna(how='any')
d.to_csv(r'abstract_without_nan.csv', index=False)

In [7]:
#authors
df=[]
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in tqdm.tqdm(f):
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            tmp_df=json.loads(cur_json[:-2])
            cur_json=''
            authors=tmp_df.get('authors')
            a_id=tmp_df.get('_id')
            if authors is not None:
                df.append((a_id, authors))
            cur_json=''
        if line == '}\n':
            tmp_df=json.loads(cur_json)
            cur_json=''
            authors=tmp_df.get('authors')
            a_id=tmp_df.get('_id')
            if authors is not None:
                df.append((a_id, authors))
            cur_json=''
            
authors=[]

for i in tqdm.tqdm(range(len(df))):
    for d in df[i][1]:
        d.update({'article_id': df[i][0]})
        authors.append(d)

409129302it [17:49, 382693.78it/s]
100%|███████████████████████████████████████████████████████████████████████| 5341234/5341234 [00:37<00:00, 140611.66it/s]


In [13]:
%%time
authors_df=pd.DataFrame(authors)
authors_df.to_csv(r'authors_raw.csv', index=False)

CPU times: total: 2min 8s
Wall time: 4min 57s


In [12]:
#keywords

df=[]
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in tqdm.tqdm(f):
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            tmp_df=json.loads(cur_json[:-2])
            cur_json=''
            keywords=tmp_df.get('keywords')
            a_id=tmp_df.get('_id')
            if keywords is not None:
                df.append((a_id, keywords))
            cur_json=''
        if line == '}\n':
            tmp_df=json.loads(cur_json)
            cur_json=''
            keywords=tmp_df.get('keywords')
            a_id=tmp_df.get('_id')
            if keywords is not None:
                df.append((a_id, keywords))
            cur_json=''
            

keywords=[]
for i in tqdm.tqdm(range(len(df))):
    if df[i] is not None:
        for j in df[i][1]:
            keywords.append({'article_id': df[i][0], 'keywords': j})        

409129302it [16:14, 419845.64it/s]
  0%|                                                                                         | 1/4182722 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'update'

In [23]:
%%time
keywords_df=pd.DataFrame(keywords)
keywords_df.to_csv(r'keywords_raw.csv', index=False)

CPU times: total: 1min 4s
Wall time: 1min 10s


In [27]:
%%time
#references

df=[]
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in tqdm.tqdm(f):
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            tmp_df=json.loads(cur_json[:-2])
            cur_json=''
            references=tmp_df.get('references')
            a_id=tmp_df.get('_id')
            if references is not None:
                df.append((a_id, references))
            cur_json=''
        if line == '}\n':
            tmp_df=json.loads(cur_json)
            cur_json=''
            references=tmp_df.get('references')
            a_id=tmp_df.get('_id')
            if references is not None:
                df.append((a_id, references))
            cur_json=''
 

409129302it [16:33, 411621.30it/s]

CPU times: total: 16min 33s
Wall time: 16min 33s


In [28]:
references=[]
for i in tqdm.tqdm(range(len(df))):
    if df[i] is not None:
        for j in df[i][1]:
            references.append({'article_id': df[i][0], 'references': j})        


100%|███████████████████████████████████████████████████████████████████████| 4004432/4004432 [00:35<00:00, 112674.96it/s]


In [30]:
%%time
references_df=pd.DataFrame(references)
references_df.to_csv(r'references_raw.csv', index=False)

In [32]:
%%time
#fos

df=[]
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in tqdm.tqdm(f):
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            tmp_df=json.loads(cur_json[:-2])
            cur_json=''
            fos=tmp_df.get('fos')
            a_id=tmp_df.get('_id')
            if fos is not None:
                df.append((a_id, fos))
            cur_json=''
        if line == '}\n':
            tmp_df=json.loads(cur_json)
            cur_json=''
            fos=tmp_df.get('fos')
            a_id=tmp_df.get('_id')
            if fos is not None:
                df.append((a_id, fos))
            cur_json=''
 

409129302it [17:01, 400647.34it/s]

CPU times: total: 16min 23s
Wall time: 17min 7s


In [36]:
fos=[]

for i in tqdm.tqdm(range(len(df))):
    if df[i] is not None:
        for j in df[i][1]:
            fos.append({'article_id': df[i][0], 'fos': j})        

100%|████████████████████████████████████████████████████████████████████████| 5003384/5003384 [02:11<00:00, 38179.72it/s]


In [37]:
%%time
fos_df=pd.DataFrame(fos)
fos_df.to_csv(r'fos_raw.csv', index=False)

CPU times: total: 1min 35s
Wall time: 5min 14s


In [44]:
#venue

df=[]
with open('dblpv13.json', encoding='utf-8') as f:
    cur_json=''
    for line in tqdm.tqdm(f):
        if line =='[\n' or line == ']': 
            continue
        cur_json+=re.sub(r"NumberInt\((\d+)\)", r"\1", line)
        if line == '},\n':
            tmp_df=json.loads(cur_json[:-2])
            cur_json=''
            venue=tmp_df.get('venue')
            a_id=tmp_df.get('_id')
            if venue is not None:
                df.append((a_id, venue))
            cur_json=''
        if line == '}\n':
            tmp_df=json.loads(cur_json)
            cur_json=''
            venue=tmp_df.get('venue')
            a_id=tmp_df.get('_id')
            if venue is not None:
                df.append((a_id, venue))
            cur_json=''

409129302it [17:09, 397294.73it/s]


In [52]:
venue=[]

for d in tqdm.tqdm(df):
    d[1].update({'article_id': d[0]})
    venue.append(d[1])

100%|██████████████████████████████████████████████████████████████████████| 5347732/5347732 [00:05<00:00, 1010035.22it/s]


In [53]:
%%time
venue_df=pd.DataFrame(venue)
venue_df.to_csv(r'venue_raw.csv', index=False)

CPU times: total: 25.7 s
Wall time: 26.4 s


In [59]:
authors_df[(authors_df['_id'].isnull()) & (authors_df['name'].isnull())]

,_id,name,article_id,org,gid,oid,orgid,sid,email,orgs,orcid,bio,name_zh,org_zh,oid_zh,orgs_zh,position,avatar,homepage
59,NaN,NaN,53e99784b7602d9701f3f913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2449,NaN,NaN,53e99785b7602d9701f44d0a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3085,NaN,NaN,53e9978ab7602d9701f45ba7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3086,NaN,NaN,53e9978ab7602d9701f45ba8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5676,NaN,NaN,53e9978db7602d9701f514c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13492822,NaN,NaN,5c8f7a5b4895d9cbc64b9511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13496921,NaN,NaN,5c8f8bbf4895d9cbc651089a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13496922,NaN,NaN,5c8f8bbf4895d9cbc651089a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13496923,NaN,NaN,5c8f8bbf4895d9cbc651089a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
len(authors_df)

16478979

In [ ]:
article_map_authors=authors_df[['article_id', '_id']]

In [307]:
for item in x:
    a_id=item[0]
    tmp=pd.DataFrame(item[1])

In [ ]:
root
title: string (nullable = true)
lang: string (nullable = true)
n_citation: double (nullable = true)
page_end: string (nullable = true)
page_start: string (nullable = true)
pdf: string (nullable = true)
volume: string (nullable = true)
year: double (nullable = true)
_id: string (nullable = true)
abstract: string (nullable = true)
isbn: string (nullable = true)
issn: string (nullable = true)
issue: string (nullable = true)
doi: string (nullable = true)
fos: array (nullable = true)
  element: string (containsNull = true)

keywords: array (nullable = true)
  element: string (containsNull = true)

references: array (nullable = true)
  element: string (containsNull = true)

url: array (nullable = true)
  element: string (containsNull = true)
venue: struct (nullable = true)
  _id: string (nullable = true)
  issn: string (nullable = true)
  name: string (nullable = true)
  name_d: string (nullable = true)
  name_s: string (nullable = true)
  online_issn: string (nullable = true)
  publisher: string (nullable = true)
  raw: string (nullable = true)
  raw_zh: string (nullable = true)
  sid: string (nullable = true)
  src: string (nullable = true)
  t: string (nullable = true)
  type: long (nullable = true)
authors: array (nullable = true)
  element: struct (containsNull = true)
    _id: string (nullable = true)
    avatar: string (nullable = true)
    bio: string (nullable = true)
    email: string (nullable = true)
    gid: string (nullable = true)
    homepage: string (nullable = true)
    name: string (nullable = true)
    name_zh: string (nullable = true)
    oid: string (nullable = true)
    oid_zh: string (nullable = true)
    orcid: string (nullable = true)
    org: string (nullable = true)
    org_zh: string (nullable = true)
    orgid: string (nullable = true)
    orgs: array (nullable = true)
      element: string (containsNull = true)
    orgs_zh: array (nullable = true)
      element: string (containsNull = true)
    position: string (nullable = true)
    sid: string (nullable = true)

In [5]:
dd = authors_df.describe(include = 'all')

dd.loc['dtype'] = authors_df.dtypes
dd.loc['size'] = len(authors_df)
dd.loc['% null'] = authors_df.isnull().mean()
dd

,_id,name,article_id,org,gid,oid,orgid,sid,email,orgs,orcid,bio,name_zh,org_zh,oid_zh,orgs_zh,position,avatar,homepage
count,13024925,16472553,16478979,9882585,4860971,115792,7903527,3476091.0,2117650,5731508,979509,407134,2603,3904,233,1501,0.0,38772,3
unique,3307110,3848566,5320329,2848576,1233740,58893,27506,1923451.0,1076236,2071643,295505,395902,2364,1612,75,690,NaN,38769,3
top,54055927dabfae8faa5c5dfa,Wei Wang,5c93685ce1cd8e540c910df1,Corresponding author.,5b86c1e0e1cd8e14a39ceaf3,544bd98645ce266baee5110d,5f71b28b1c455f439fe3c989,2116008403.0,0,[],0000-0003-3133-7119,Authors’ photographs and biographies not avail...,于戈,国家卫星气象中心,544bd8f945ce266bae9984d1,['Department of Computer Science and Technolog...,NaN,/mediastore_new/IEEE/content/freeimages/727486...,http://www.maths.uq.edu.au/~bab/
freq,1629,4573,427,53458,9903,324,120338,224.0,877,364461,462,2898,7,39,19,34,NaN,2,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
5